In [1]:
import pandas as pd
import numpy as np

import sandy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
endf6_file = "dec-027_Co_060.endf"

In [3]:
tape = sandy.Endf6.from_file(endf6_file)

In [4]:
tape.to_series()

MAT  MF  MT 
558  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          0          0  ...
559  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          1          1  ...
Name: dec-027_Co_060.endf, dtype: object

The object representation uses method `.to_series` under the hood.

tape.data

In [5]:
tape.mt
tape.mat
tape.mf

[451, 457]

[558, 559]

[1, 8]

In [6]:
with open(endf6_file) as f:
    text = f.read()
tape1 = sandy.Endf6.from_text(text)

In [7]:
assert tape1.data == tape.data

In [8]:
tape.filter_by(listmat=[558])
tape.filter_by(listmf=[8], listmt=[457])
tape.filter_by(listmf=[1, 8])

MAT  MF  MT 
558  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          0          0  ...
Name: dec-027_Co_060.endf, dtype: object

MAT  MF  MT 
558  8   457     2.706000+4 5.941896+1          0          0  ...
559  8   457     2.706000+4 5.941896+1          1          1  ...
Name: dec-027_Co_060.endf, dtype: object

MAT  MF  MT 
558  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          0          0  ...
559  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          1          1  ...
Name: dec-027_Co_060.endf, dtype: object

In [9]:
new_section = "This is a new section to be added to an ENDF-6 file. Instead of text there should be fancy numbers."
tape.add_section(mat=558, mf=4, mt=999, text=new_section)

MAT  MF  MT 
558  1   451     2.706000+4 5.941896+1         -1          0  ...
     4   999    This is a new section to be added to an ENDF-6...
     8   457     2.706000+4 5.941896+1          0          0  ...
559  1   451     2.706000+4 5.941896+1         -1          0  ...
     8   457     2.706000+4 5.941896+1          1          1  ...
Name: dec-027_Co_060.endf, dtype: object

In [13]:
title = "My new file"
string = tape.write_string(title=title)
print(string)

My new file                                                          1 0  0    0
 2.706000+4 5.941896+1         -1          0          0          1 558 1451    1
 0.000000+0 1.000000+0          0          0          0          6 558 1451    2
 0.000000+0 0.000000+0          1          0          4          7 558 1451    3
 0.000000+0 0.000000+0          0          0         32          2 558 1451    4
 27-Co- 60  BNL        EVAL-NOV05 Conversion from ENSDF            558 1451    5
 /ENSDF/                                               20111222    558 1451    6
----ENDF/B-VII.1      Material  558                                558 1451    7
-----RADIOACTIVE DECAY DATA                                        558 1451    8
------ENDF-6 FORMAT                                                558 1451    9
*********************** Begin Description ***********************  558 1451   10
**         ENDF/B-VII.1 RADIOACTIVE DECAY DATA FILE            **  558 1451   11
**         Produced at the N